# SQLite Progress Tracker

The purpose of this notebook is to track student progress on their personalized 7 day study plan. It expands on Notebook 03.5 (prototype) by implementing a functional SQLite database that stores tasks,  
tracks completion, and enforces rules around daily progress.

**What this notebook does**
1. Loads or creates the `progress.db` SQLite database.  
2. Defines tables for:
   - **Plans** (plan_id, student_feedback, created_date)  
   - **Tasks** (task_id, plan_id, day_number, description, citation, est_mins, is_completed)  
3. Inserts tasks generated from Notebook 03 into the database.  
4. Provides functions to:
   - Mark tasks as complete.  
   - Query incomplete tasks.  
   - Prevent a student from advancing to the next day until all current day tasks are complete.  
5. Exports reports of progress (completed vs. remaining tasks).  

**Inputs**
- JSON-like study plan output  

**Outputs**
- `progress.db` (SQLite database with plans and tasks).  
- Example queries that show student progress across 7 days.  